detect -> convoluted -> generate -> knn -> train -> test

In [ ]:
!python3 -m pip install -r requirements.txt

In [ ]:
import cv2
import numpy as np

# Used to pad around extracted faces
offset = 10

# Initialize haar cascade filters provided by
# https://github.com/opencv/opencv/tree/master/data/haarcascades
face_cascade = cv2.CascadeClassifier("./haarcascades/haarcascade_frontalface_default.xml")

# Crop faces from photos
def crop(photo_path):
    
    # Read Photo into the img var
    img = cv2.imread(photo_path)
    # cv2.imshow('photo', img)
 
    # Apply grayscale to the image
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
 
    # Face Detection
    # Cascading params: image, scaleFactor, minNeighbors
    face = face_cascade.detectMultiScale(gray, 1.3, 5)
 
    if len(face) == 0:
        return face
 
    x,y,w,h = face[0]
 
    # Drawing a rectangle around the face coordinates
    # cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2) 
 
    # Slicing face from original image
    face_cut = img[y-offset:y+h+offset,x-offset:x+w+offset]
    if 0 in face_cut.shape:
        return []
    # Resizing faces to 128x128px
    face_cut = cv2.resize(face_cut,(128,128))

    # Display each face and wait for keypress before proceeding
    # cv2.imshow('crop', face_cut)
    # cv2.waitKey(0)
 
    return face_cut

In [ ]:
def edgy_af(img):
    """
    Apply crop, edge detection, and grayscale filters to input image
    """

    # Apply edge detection kernel
    h_kernel = np.array(
        [
            [-1, 0, 1],
            [-1, 0, 1],
            [-1, 0, 1],
        ]
    )
    v_kernel = np.array(
        [
            [-1, -1, -1],
            [0, 0, 0],
            [1, 1, 1],
        ]
    )
    img = cv2.filter2D(src=img, ddepth=-1, kernel=h_kernel) + cv2.filter2D(
        src=img, ddepth=-1, kernel=v_kernel
    )

    # Make grayscale
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    return img


In [ ]:
import os

# Parse through a dataset to prepare face analysis
img_dict = {}
face_count = 0

train_dir = "samples"
dataset_path = "dataset/"

for f_name in os.listdir(train_dir):
    for p_img in os.listdir(os.path.join(train_dir, f_name)):
        face_count += 1
        if f_name in img_dict.keys():
            img_dict[f_name].append(os.path.join(train_dir, f_name, p_img))
        else:
            img_dict[f_name] = [os.path.join(train_dir, f_name, p_img)]

# Metrics for counting detected faces
detect_count = 0

# Looping through all image directory values
for person in img_dict:

    # Create a new array for person's images
    face_data = []

    for photo in img_dict[person]:

        print(photo)

        face_cut = crop(photo)

        if len(face_cut) == 0:
            continue

        face_filter = edgy_af(face_cut)

        detect_count += 1

        # Add face data to array for dataset export
        face_data.append(face_filter)

    # Save the dataset
    if len(face_data) == 0:
        continue

    # Converting data to np array and saving to /data
    face_data = np.asarray(face_data)
    face_data = face_data.reshape((face_data.shape[0], -1))
    np.save(dataset_path + person + ".npy", face_data)

    print("Saved " + person + "'s data to /data")

print("Detected " + str(detect_count / face_count * 100) + "%")
print("(" + str(detect_count) + "/" + str(face_count) + ")")


In [ ]:
# Find distance between two euclidian values
def distance(v1, v2):
    return np.sqrt(((v1-v2)**2).sum())

def predict(train, test, k=3):
    dist = []

    for i in range(train.shape[0]):
        # Get vector and label
        ix = train[i, :-1]
        iy = train[i, -1]
        print(test)
        print(ix)
        # Computing the distance from the test point
        d = distance(test, ix)
        dist.append([d,iy])
    # Sort based on distance and get top k
    dk = sorted(dist, key=lambda x: x[0])[:k]
    # Retrieve only the labels
    labels = np.array(dk)[:, -1]

    # Get frequencies of each label
    output = np.unique(labels, return_counts=True)
    # Find max frequency and corresponding label
    index = np.argmax(output[1])
    return output[0][index]


In [ ]:
import json

dataset_path = 'dataset/'

face_data = []
labels = []

class_id = 0 # Labels for the given file
names = {} # Mapping id & name

# Data Prep
for fx in os.listdir(dataset_path):

    if fx.endswith('.npy'):

        # Create class_id & name mapping
        names[class_id] = fx[:-4] # index removes .npy from name
        print('Loaded ' + fx)

        data_item = np.load(dataset_path+fx)
        face_data.append(data_item)

        # Create Labels for the class
        target = class_id*np.ones((data_item.shape[0],))
        class_id += 1
        labels.append(target)

face_dataset = np.concatenate(face_data,axis=0)
face_labels = np.concatenate(labels,axis=0).reshape((-1,1))

print(face_dataset.shape)
print(face_labels.shape)

trainset = np.concatenate((face_dataset,face_labels),axis=1)

# Saving label names
json_str = json.dumps(names)
f = open('names.json', 'w')
f.write(json_str)
f.close()

# Saving model
np.save('model.npy', trainset)

print("Model finished training and saved to model.npy.\nShape:\n")

print(trainset.shape)


In [ ]:
test_dir = 'tests'

# Loading the datasets
trainset = np.load('model.npy')
 
f = open('names.json')
names = json.load(f)
f.close()

for f_name in os.listdir(test_dir):
    photo_path = test_dir + '/' + f_name
    face_cut = crop(test_dir + '/' + f_name)

    if len(face_cut) == 0:
        print(f"Face not detected in test image: {photo_path}.\n")
        continue

    face_filter = edgy_af(face_cut)

    # Make face prediction
    out = predict(trainset,face_filter.flatten())
    pred_name = names[str(int(out))];

    print("Image " + photo_path + " is predicted to be: ")
    print(pred_name + "\n")

    cv2.imshow('tests', face_filter)
    cv2.waitKey(0)

cv2.destroyAllWindows()
